# Virtual Environment Configuration

In [ ]:
%pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [ ]:
!conda create -p env python=3.8 -y

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /content/env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    libffi-3.4.6               |       h2dba641_0          52 KB  conda-forge
    libgcc-14.2.0              |       h767d61c_2         828 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_2          52 KB  conda-forge
    libgomp-14.2.0             |       h767d61c_2         449 KB  conda-forge
    liblzma-5.6.

In [ ]:
%%writefile requirements.txt

bentoml==1.0.25
scikit-learn
cattrs==23.1.1

Writing requirements.txt


In [ ]:
!source activate ./env; pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 142.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.5/746.5 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 30.4 MB/s eta 0:00:00


In [ ]:
!source activate ./env; pip install bentoml==0.9.0 --quiet
# !source activate ./env; pip install bentoml --quiet
# !source activate ./env; pip install bentoml -U --quiet 

In [ ]:
!source activate ./env; pip install -U scikit-learn scipy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 77.5 MB/s eta 0:00:00


---

# BentoML implementation 

In [ ]:
%%writefile bento_train.py

import bentoml

from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
iris = load_iris()
X = iris.data[:, :4]
Y = iris.target
model.fit(X, Y)

bento_model = bentoml.sklearn.save_model('kneighbors', model)
print(f"Model saved: {bento_model}")

Writing bento_train.py


In [ ]:
!source activate ./env; python bento_train.py

Model saved: Model(tag="kneighbors:iv5wxyxy6kw66asc")


In [ ]:
!source activate ./env; bentoml models list

 Tag                          Module           Size       Creation Time       
 kneighbors:iv5wxyxy6kw66asc  bentoml.sklearn  14.11 KiB  2025-03-04 12:14:50 


In [ ]:
%%writefile bento_test.py

import bentoml

clf = bentoml.sklearn.get('kneighbors:latest').to_runner()
clf.init_local()
print(clf.predict.run([[2,3,4,5]]))

Writing bento_test.py


In [ ]:
!source activate ./env; python bento_test.py

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.
[2]


In [ ]:
%%writefile bento_cmd.txt

bentoml models list

bentoml serve service.py:service --repload

bentoml build

bentoml list

Writing bento_cmd.txt


In [ ]:
%%writefile bentofile.yaml
service: "service.py:service"  # Same as the argument passed to `bentoml serve`
labels:
    owner: bentoml-team
    project: gallery
include:
- "*.py"  # A pattern for matching which files to include in the bento
python:
    packages:  # Additional pip packages required by the service
    - scikit-learn
    - pandas

Writing bentofile.yaml


In [ ]:
%%writefile service.py

import bentoml
import numpy as np
from bentoml.io import NumpyNdarry

clg = bentoml.sklearn.get('kneighbors:latest').to_runner()

service = bentoml.Service("kneighbors", runners=[clf])

# Create an API function
@service.api(input=NumpyNdarray(), output=NumpyNdarray())
def predict(input_series: np.ndarray) -> np.ndaray:

   result = clf.predict.run(input_series)

   return result

Writing service.py


In [ ]:
!source activate ./env; bentoml serve kneighbors:latest

Error: [bentoml-cli] `serve` failed: Failed to load bento or import service 'kneighbors:latest'.
If you are attempting to import bento in local store: 'Failed to import module "kneighbors": No module named 'kneighbors''.
If you are importing by python module path: 'no Bentos with name 'kneighbors' exist in BentoML store <osfs '/root/bentoml/bentos'>'.


In [ ]:
!source activate ./env; bentoml build

Error: [bentoml-cli] `build` failed: Failed to import module "service": cannot import name 'NumpyNdarry' from 'bentoml.io' (/content/env/lib/python3.8/site-packages/bentoml/io.py)


以上錯誤，顯示無法在 Colab 上運行此 project